In [25]:
import json
import matplotlib.pyplot as plt
import keras
import keras.layers as layers
from GHCND import *

## Retrieve data

To filter the stations used, only stations with fewer than 100 missing data points were considered. The number of missing values is calculated in the script `get_station_counts.py` (took ~50 minutes to run on my laptop). These counts are stored in the file `stat_counts_tmax.json` in the json format for ease of future loading, as this script need only be run once for a given set of stations and data. The file is opened and read below, then all stations with 100 or fewer missing data points saved in a seperate list. 

In [26]:
# open json file containing the number of data gaps for the stations
f = open('data/stat_counts_tmax.json')
data = json.load(f)

# find all stations with no data gaps
no_gaps_tmax = [name for name, count in data.items() if count <= 100]
print(f"Number of stations found: {len(no_gaps_tmax)}")


Number of stations found: 103
ASN00003003


Retrieve data for the given station and variable. An instance of the GHCND class is created, then the `readCountriesFile` and `readStationsFile` methods called on it. These methods extract information on the countries and stations available and stores these in fields as dictionaries. The list of station names is stored as a seperate list for future use.

The station to be investigated is set from the list of those with few missing date points. The required file name is constructed from the station name, which is then used to build the url required to fetch the data from the course-provided web directory of data files. The required data file is then copied from this remote directory to a local one (`/data`). The `processFile` method is then called on the GHCND class instance to extract the data from this file into a dictionary.

Data for the variable specified is extracted from this dictionary into an instance of the `Variable` class, with fields for the values and their corresponding dates.

In [28]:
# create instance of the GHCND class and extract information on countries and stations from their respective files
ghn = GHCND()
ghn.readCountriesFile()
ghn.readStationsFile()

# get list of station names
station_names = ghn.getStatKeyNames()

# set station to be investigated
station = no_gaps_tmax[0]
print(f"Station selected: {station}")

# get url for a specified station
fileName = f"{station}.dly"
print(f"Filename: {fileName}")
urlName = f"http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/ghcnd/ghcnd_gsn/{fileName}"
print(f"url name: {urlName}")

# copy station data from remote to local
destination = f"data/stations_daily/{fileName}"
print(f"destination: {destination}")
urllib.request.urlretrieve(urlName, destination)
station_data = ghn.processFile(destination)
print(f"Station details: {ghn.getStation(station)}")

# extract data for specified variable into an instance of the Variable class
t_max = Variable(ghn.getVar(station_data, 'TMAX'), "max temp (degC)", ghn.stationDict[station].name)
dates = t_max.get_dates()
vals = t_max.get_vals()


Read 219 countries and codes
Read 991 stations with justGSN
Station selected: ASN00003003
Filename: ASN00003003.dly
url name: http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/ghcnd/ghcnd_gsn/ASN00003003.dly
destination: data/ASN00003003.dly
Station details: ASN00003003 is BROOME AIRPORT, Australia at -17.9475 122.2353 7.4


## Prepare data for training

As this section is concerend with the climate at a given station rather than the weather, the monthly mean for the specified variable is found by calling the `get_monthly_means` method on the instance of the Variable class. To improve the performance and training stavility of the model, he date are then normalised by calling the `normalise` method, which subtracts the mean of the entire dataset then subtracts the maximum value for each data point. The first ten months of these mean values and normalised mean values are plotted.

In [ ]:
# get mean values for each month of data
means = np.array(t_max.get_monthly_means())
print(f"Number of months of data: {len(means)}")

# normalise means
means_normalised = t_max.normalise(means)

# plot means and normalised means
fig, ax = plt.subplots()
ax.plot(means[:12*10])
ax.set_xlabel("Month")
ax.set_ylabel("Mean monthly maximum temperature (degC)")
ax.set_title("Mean and normalised mean monthly maximum temperatures over 10 years")
ax2 = ax.twinx()
ax2.plot(means_normalised[:(12*10)])
ax2.set_ylabel("Normalised mean monthly maximum temperature")


The data are divided into training, validating, and testing datasets. The training dataset must be the largest in order to provide sufficient material for the model to achieve a good performance. The testing dataset is smaller, but large enough that the model has sufficient input for predicting future sequences. The validation dataset is the smallest. A training:testing:validation ratio of 0.7:0.2:0.1 is used.

The offset describes how far into the future the model will predict. An offset of 12 is used here, meaning that the model will predict twelve months ahead. The window size dictates how much data the model is trained on in a single batch. If this is too small, the model will not perform well, however it must be smaller than the validation dataset minus the offset, otherwise it will be impossible to shape the dataset as necessary in later steps. It was found during testing that a window size of 80 worked well and gave a good performance.

The divided datasets are then split into a series of overlapping windows with length `WINDOW_SIZE`, and an associated value `OFFSET` points further ahead in the time series. These are the input windows and targets on which the model will be trained, validated and tested.

These windows are reshaped from (number of windows, window size) to (number of windows, window size, number of features), where 'number of features' is the number of variables being used in this model's training, in this case, one. 

In [24]:
OFFSET = 12

# calculate appropriate divisions of data
test_len = int(len(means_normalised) * 0.2)
train_len = int(len(means_normalised) * 0.7)
validate_len = int(len(means_normalised) * 0.1)

# if the validation dataset is too small, use a smaller window size
if validate_len < 80 + OFFSET + 1:
    WINDOW_SIZE = validate_len - OFFSET - 1
else:
    WINDOW_SIZE = 80

print(f"Number of months of training data: {train_len}")
print(f"Number of months of validation data: {validate_len}")
print(f"Number of months of testing data: {test_len}")

# divide data into training, validating and testing sets
means_test = means_normalised[:test_len]
means_train = means_normalised[test_len+1:test_len+train_len]
means_validate = means_normalised[test_len+train_len+1:]

# split data into input windows and targets
input_test, target_test = shapeArray(means_test, WINDOW_SIZE, OFFSET)
input_train, target_train = shapeArray(means_train, WINDOW_SIZE, OFFSET)
input_validate, target_validate = shapeArray(means_validate, WINDOW_SIZE, OFFSET)

# reshape the data into the correct format for input into the model
n_features = 1
input_train = input_train.reshape((input_train.shape[0], input_train.shape[1], n_features))
input_test = input_test.reshape((input_test.shape[0], input_test.shape[1], n_features))
input_validate = input_validate.reshape((input_validate.shape[0], input_validate.shape[1], n_features))


Number of months of training data: 1089
Number of months of validation data: 155
Number of months of testing data: 311
input shape: (220, 80); label shape: (220, 12)
input shape: (997, 80); label shape: (997, 12)
input shape: (64, 80); label shape: (64, 12)


## Train model

Larger window size makes the loss larger at the beginning, but makes the validation loss closer to the training loss - better at predicting unseen data.
Good window size: 80 

In [ ]:
model = keras.models.Sequential()
model.add(layers.LSTM(64, input_shape = (WINDOW_SIZE, 1), activation = 'relu', return_sequences = False))
model.add(layers.Dense(1, activation = "linear"))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

model.summary()

# train model and extract final loss
history = model.fit(input_train, target_train, epochs = 100, validation_data = (input_validate, target_validate))
cost = history.history['loss']
val_cost = history.history['val_loss']

fig, ax = plt.subplots()
ax.plot(cost)
ax.plot(val_cost)

In [ ]:
prediction = model.predict(input_test)

fig, ax = plt.subplots()
ax.plot(prediction)
ax.plot(target_test)

### Plot first 10 years of data against predictions